<a href="https://colab.research.google.com/github/FernandoBRdgz/inteligencia_artificial/blob/main/grandes_modelos_de_lenguaje/introducci%C3%B3n_a_llms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introducción

En la era digital actual, el procesamiento del lenguaje natural ha experimentado avances significativos gracias a los modelos de aprendizaje automático, en particular, los modelos largos del lenguaje. Estos modelos, basados en técnicas de inteligencia artificial, han revolucionado la forma en que las computadoras comprenden y generan texto de manera similar a los humanos.

Un modelo largo del lenguaje es capaz de aprender patrones complejos y estructuras gramaticales en grandes cantidades de texto, adquiriendo conocimiento sobre el lenguaje en general. Estos modelos se entrenan en enormes conjuntos de datos, como libros, artículos de noticias, páginas web y corpus lingüísticos, para capturar las regularidades y matices del lenguaje humano.

### Funcionamiento

La arquitectura principal detrás de estos modelos es el Transformer, que se destaca por su capacidad para procesar secuencias de texto de manera eficiente. El Transformer utiliza capas de atención y cálculos paralelos para capturar las relaciones entre las palabras y generar representaciones contextualizadas de cada palabra en función de su contexto. Estas representaciones son fundamentales para la capacidad del modelo de comprender el significado y la estructura del texto.

Una vez que un modelo largo del lenguaje ha sido entrenado, puede ser utilizado para diversas tareas del procesamiento del lenguaje natural, como la traducción automática, la generación de texto, el resumen de documentos, el análisis de sentimientos y la respuesta a preguntas. Estos modelos son altamente flexibles y pueden adaptarse a diferentes dominios o tareas específicas.

### Limitaciones

Si bien los modelos largos del lenguaje han demostrado ser asombrosamente efectivos para muchas aplicaciones, también presentan desafíos. El entrenamiento y la utilización de estos modelos requieren una gran cantidad de recursos computacionales y de almacenamiento, así como una gran cantidad de datos de entrenamiento etiquetados. Además, la ética y la privacidad son consideraciones importantes en el uso de estos modelos, ya que pueden generar contenido potencialmente engañoso o discriminatorio si no se manejan adecuadamente. Y dificultades para comprender el razonamiento y la causalidad (sufren de alucinaciones).

## Arquitectura base (El Transformador)

En 2017, un equipo de Google publicó el artículo “Atención es todo lo que necesitas” introduciendo una nueva arquitectura a la cual nombraron Transformer. La idea principal es detener el uso de RNNs y CNNs y, en lugar, se utilizara esta arquitectura (Fig. 2), con el uso de mecanismos de atención pero de una manera muy ingeniosa. Esto les permitió obtener modelos mucho más paralelizables y, por lo tanto, computacionalmente eficientes comparado con arquitecturas como LSTMs o GRUs mostrando que los Transformadores modelan de mejor forma las dependencias de largo plazo entre las palabras. Puedes ver la arquitectura aquí y cómo funciona en el nivel más alto.

![Arquitectura Transformer](https://d2l.ai/_images/transformer.svg)

In [ ]:
!pip install openai tiktoken faiss-cpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.6/17.6 MB 69.1 MB/s eta 0:00:00


In [ ]:
!pip install langchain==0.0.154 PyPDF2==3.0.1 python-dotenv==1.0.0 streamlit==1.18.1 gradio==3.28.3 gradio_client==0.1.4 requests==2.27.1 gTTS==2.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from dotenv import load_dotenv
import gradio as gr
import io
import requests
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.callbacks import get_openai_callback
from gtts import gTTS

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
import os
import sys

In [ ]:
path = '/content/gdrive/MyDrive/Colab Notebooks/Inteligencia Artificial/Q&A/'

In [ ]:
sys.path.append(path)

In [ ]:
from config import api_key_openai

In [ ]:
os.environ["OPENAI_API_KEY"] = api_key_openai

In [ ]:
def get_pdf_from_url(url):
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Windows; Windows x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.114 Safari/537.36'}
    response = requests.get(url=url, headers=headers, timeout=120)
    with open('document.pdf', 'wb') as file:
        file.write(response.content)
    return 'document.pdf'

In [ ]:
def questiondocument(user_question, url):
    load_dotenv()
    document = get_pdf_from_url(url)
    pdf_reader = PdfReader(document)
    text = ""
    for page in pdf_reader.pages:
      text += page.extract_text()
        
    text_splitter = CharacterTextSplitter(
      separator="\n",
      chunk_size=1000,
      chunk_overlap=200,
      length_function=len
    )
    chunks = text_splitter.split_text(text)
      
    embeddings = OpenAIEmbeddings()
    knowledge_base = FAISS.from_texts(chunks, embeddings)
      
    docs = knowledge_base.similarity_search(user_question)
        
    llm = OpenAI()
    chain = load_qa_chain(llm, chain_type="stuff")
    with get_openai_callback() as cb:
      response = chain.run(input_documents=docs, question=user_question)
    audio = gTTS(response, lang='es')
    audio.save("output.mp3")

    cost = cb
    return response, cost, "output.mp3"

In [ ]:
demo = gr.Interface(
    fn=questiondocument,
    inputs=["text", "text"],
    outputs=["text","text","audio"],
)
demo.launch(share=True, debug=True)

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://87a271e8b9946820e3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://87a271e8b9946820e3.gradio.live


**Referencias**

* https://platform.openai.com/docs/models/overview